In [20]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_userprofile(self):
        
        def get_desc(desc_set, colLength, count):
            count = desc_set.shape[1]
            while(count < colLength):
                desc_set = np.append(desc_set, 1)
                count += 1
            
            return desc_set
        
        # Source: https://stackoverflow.com/questions/1518522/python-most-common-element-in-a-list
        # by user Alex Martelli
        def most_common(L):
            # get an iterable of (item, iterable) pairs
            SL = sorted((x, i) for i, x in enumerate(L))
            # print 'SL:', SL
            groups = itertools.groupby(SL, key=operator.itemgetter(0))
            # auxiliary function to get "quality" for an item
            def _auxfun(g):
                item, iterable = g
                count = 0
                min_index = len(L)
                for _, where in iterable:
                    count += 1
                min_index = min(min_index, where)
                # print 'item %r, count %r, minind %r' % (item, count, min_index)
                return count, -min_index
            # pick the highest-count/earliest item
            result = max(groups, key=_auxfun)[0]
            return result
    
        # Source: https://stackoverflow.com/questions/32037893/numpy-fix-array-with-rows-of-different-lengths-by-filling-the-empty-elements-wi
        # by user Divakar
        def numpy_fillna(data):
            # Get lengths of each row of data
            lens = np.array([len(i) for i in data])
    
            # Mask of valid places in each row
            mask = np.arange(lens.max()) < lens[:,None]
    
            # Setup output array and put elements from data into masked positions
            out = np.zeros(mask.shape, dtype=data.dtype)
            out[mask] = np.concatenate(data)
            return out    
    
        def tokenize(s):
            return tokens_re.findall(s)
    
            #args s = tweets, lowercase=False(keeps as in tweets) / True(changes all to lowercase)
        def preprocess(s, lowercase=True):
            tokens = tokenize(s)
            if lowercase:
                tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
            return tokens
        
        from sklearn.svm import LinearSVC
        from nltk.corpus import stopwords
        import pickle
        import re
        import string

        import numpy as np
        import tweepy
        import operator
        import itertools
        import warnings
        
        warnings.filterwarnings('ignore')
        
        #Auth details from Twitter app http://twitclassify.com/
        CONSUMER_KEY = "90bt0hlKzf3zWi5aPJ5ELZOGL"
        CONSUMER_SECRET = "H4ycskJVRjcesIaZBVUFdTClsucdMEyPcZyQT7Xjvou3LWwBg2"
        ACCESS_KEY = "303590178-hw3mgzNW08xAvbrVW4Zm8VP4uKpU9qyfG01ynkzi"
        ACCESS_SECRET = "6wJw3hGpFAgR77nk6Hw4UjCGi96OtOZhnXjm1iMcptQ3i"
        
        authid = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        authid.set_access_token(ACCESS_KEY, ACCESS_SECRET)
        myApi = tweepy.API(authid, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        
        punctuation = list(string.punctuation)
        stop = stopwords.words('english') + punctuation + ['rt', 'via', 'if', 'i', '’', '“', '”', 'it', 'the', '…', 'us', ':)', '—',
                                                           ':/', 'il', '', '≠', '•']
        
        # Source taken from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
        emoticons_str = r"""
            (?:
                [:=;] # Eyes
                [oO\-]? # Nose (optional)
                [D\)\]\(\]/\\OpP] # Mouth
            )"""
         
        regex_str = [
            emoticons_str,
            r'<[^>]+>', # HTML tags
            r'(?:@[\w_]+)', # @-mentions
            r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
            r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
         
            r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
            r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
            r'(?:[\w_]+)', # other words
            r'(?:\S)' # anything else
        ]
            
        tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
        emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
           
        ### Lists ###
        desc_set = []
        li = []
        convList = []
        
        twitHandle = ['sdrogers']
 
        user_objects = myApi.lookup_users(screen_names= twitHandle)
        user_id = [user.id_str for user in user_objects]
        userProfile = myApi.lookup_users(user_ids=user_id)
        
        for user in userProfile:
           
            #user profile description
            terms_stop = [term for term in preprocess(user.description) 
                          if term not in stop and
                          not term.startswith(('#', '@', 'http', 'https'))]
            
            # separates each descriptions to their respective users
            # add list of desc for each user into desc train set ie. desc_train_set = [[desc 1][desc 2]...[desc n]]
            desc_set.append(terms_stop)
        
        #load pickle profileDict here
        with open('profileDict.pkl', 'rb') as f:
            uniqueDict = pickle.load(f)
            
        #inv_map = {v: k for k, v in uniqueDict.items()} # to reverse dict ordering
        for index,word in enumerate(desc_set):         # iterate over set    
            for key, value in uniqueDict.items():
                if value in word:                      # see if word exists in dictionary
                    convList.append(key)
            li.append(convList)
            convList = []
    
        desc_set = np.asarray(li)
        # Match dimensions of array with highest no. of feature
        desc_set = numpy_fillna(desc_set)
    
        with open('profileColLen.pkl', 'rb') as f:
            colLength = pickle.load(f)
        count = int(desc_set.shape[1])
        
        desc = get_desc(desc_set, colLength, count)
        # Load from .pickle
        with open('profileModel.pkl','rb') as f:
            modelSVM = pickle.load(f)
            
            # make predictions
        predicted = modelSVM.predict(desc)
        # summarize the fit of the model
        # Classification report
        num = most_common(predicted)
    
        if(num == '1'):
            community = "Academic"
        elif(num == '2'):
            community = "Media"
        elif(num == '3'):
            community = "Business"
        elif(num == '4'):
            community = "Politic"
        elif(num == '5'):
            community = "Citizen"
    
        category = community        
                
        self.assertEqual(category, 'Academic')

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.640s

OK
